# Parsing
Parse json files to dataframe

In [ ]:
import zipfile
import pandas as pd
import json

df_list = []
with zipfile.ZipFile("gptsurvey_results.zip", "r") as zip_ref:
    for file in zip_ref.namelist():
        with zip_ref.open(file) as json_file:
            data = json.load(json_file)
            row = {}
            row["CourseID"] = data["CourseID"]
            row["task_name"] = data["task_name"]
            row["SciperID"] = data["SciperID"]
            for key in data["results"]:
                row[key] = str(data["results"][key])
            df = pd.DataFrame(row, index=[0])
            df_list.append(df)
df = pd.concat(df_list)
# Remove dummy scipers
df = df[~df["SciperID"].str.contains("00000")]
df = df.reset_index(drop=True)
# Hash scipers
df["SciperID"] = df["SciperID"].apply(lambda x: str(hash(x)))
df

# Duplicate results
Survey taken twice for same sciper

In [ ]:
df[df["SciperID"].duplicated(keep=False)]

# Save to csv

In [ ]:
df.to_csv("gptsurvey_results.csv", index=False)